In [1]:
!pip install transformers-interpret==0.6.0
!pip install datasets
!pip install tweet-preprocessor
!pip install emoji==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# connect to gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [53]:
import torch
import datasets
import json
import re
import os
import preprocessor as p
import torch.nn as nn
import nltk
import pandas as pd
import torch.nn.functional as F

from datasets import Dataset, load_metric, load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import get_scheduler, AdamW
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer,AutoModelForSequenceClassification, DataCollatorWithPadding,BartForConditionalGeneration
from transformers_interpret import MultiLabelClassificationExplainer,SequenceClassificationExplainer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dir_path = '/content/gdrive/MyDrive/PhD/Explainability_Project_2022/RV_models/BERT/Final_notebook/'

The prediction labels are: 

*   0 = True
*   1 = False
* 2 = *Unverified*





# 1. Construct BERT model for rumour verification

We train a BERT model using the source tweet (claim of the rumour) and first-level replies (referred as replies for convenience) as input. For the purpose of this study we ignore replies posted deeper in the thread e.g. replies of replies. 

Training instances are of the form: *source tweet [SEP] reply*. Hence, for a thread with *n* replies, we generate *n* training instances.

Note that this enables us to use all the information in the replies and avoid out-of-memory issues which occur in the rationale generation for Section 2. The trade-off is sacrificing some faithfulness as the claim is predicted *n* labels which are aggregated to form a final label.

In [6]:
#Preprocess function for tweets

p.set_options(p.OPT.EMOJI, p.OPT.SMILEY)
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = tweet.replace('\n','').replace('\t','')
    tweet = re.sub(r'http\S+', 'URL', tweet)
    tweet = re.sub(r'@\w+', 'MENTION', tweet)
    tweet = p.clean(tweet)
 
    return tweet

In [ ]:
def prep_data(fold, cvfolds):  
    # We only use the original (unshuffled) data for testing
    test = {}
    test['data'] = []
    test['data'] = cvfolds[fold]
    
    savepath1 = dir_path + "test_" + fold + ".json"
    with open(savepath1, 'w+') as f:
        json.dump(test, f)    
    testset = load_dataset('json', data_files=savepath1, field='data')
    
    train = {}
    train['data'] = []
    for fo in list(cvfolds.keys()): 
        if fo!=fold:
            train['data'].extend(cvfolds[fo])
    savepath2 = dir_path + "train_" + fold + ".json"
    with open(savepath2, 'w+') as f:
        json.dump(train, f)
    trainset = load_dataset('json', data_files=savepath2, field='data')
    
    return testset, trainset    

In [ ]:
checkpoint = "vinai/bertweet-large" # "bert-base-uncased" # "cardiffnlp/twitter-roberta-base" #"roberta-large" #"bert-base-uncased" #"digitalepidemiologylab/covid-twitter-bert-v2" #"cardiffnlp/twitter-roberta-base" # "roberta-large" # "bert-base-uncased" # # #  # # # # #" # # #"digitalepidemiologylab/covid-twitter-bert-v2" #"roberta-large"#"bert-base-uncased" #"bert-large-cased" #"roberta-large" #"bert-base-uncased" #"bert-large-cased" # #"bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
bs = 4

def tokenize_function(example):   
    return tokenizer(example['pp_source_reply'], truncation=True,max_length=512)

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

Note that the model is trained on a leave-one-event out cross validation setting.

In [ ]:
#Load data

with open(os.path.join(dir_path, 'pheme_reply_entries.json'),'r') as f:
    cvfolds = json.load(f)

for fold in list(cvfolds.keys()):
 
    testset, trainset = prep_data(fold, cvfolds)
    testset_ids = list(testset['train']['idx'])
    
    print (len(testset_ids))
  
    tokenized_datasets = trainset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "idx","source_reply","pp_source_reply"])
    tokenized_datasets.set_format("torch")
    print(tokenized_datasets)
    
    tokenized_testset = testset.map(tokenize_function, batched=True)
    tokenized_testset = tokenized_testset.remove_columns(["sentence1", "idx","source_reply","pp_source_reply"])
    tokenized_testset.set_format("torch",columns=['input_ids', 'attention_mask', 'labels']) #'token_type_ids', for bert?
    
    train_dataloader = DataLoader(
        tokenized_datasets["train"], shuffle=True, batch_size=bs, collate_fn=data_collator)
   
    eval_dataloader = DataLoader(
        tokenized_testset["train"], batch_size=bs, collate_fn=data_collator)
    
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)
    
    optimizer = AdamW(model.parameters(), lr=1e-5)
    
    num_epochs = 10
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler("linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=num_training_steps)
    #print(num_training_steps)

    progress_bar = tqdm(range(num_training_steps))

    model.train()
    for epoch in range(num_epochs):
        for batch in train_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            
    # Save state dictionary    
    model_path = dir_path'+'model_{}'.format(fold)
    torch.save(model.state_dict(), model_path)
    

    all_test_predictions = []
    all_test_labels = []
    metric = load_metric("accuracy", "f1")
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        all_test_predictions.extend(predictions)
        all_test_labels.extend(batch['labels'])
        metric.add_batch(predictions=predictions, references=batch["labels"])

    metric.compute()
    print(fold,all_test_predictions)
    all_test_predictions = [int(i) for i in all_test_predictions]
    all_test_labels = [int(i) for i in all_test_labels]
    
    print(accuracy_score (all_test_labels, all_test_predictions))
    print(f1_score(all_test_labels, all_test_predictions, average='macro'))
    
    model.apply(reset_weights) 

# 2. Apply Rationale-based explainer on BERT

Load Model trained for 'sydneysiege" using the state dictionary. 
This is found in the workspace folder.



In [5]:
  checkpoint = "vinai/bertweet-large" # "bert-base-uncased" # "cardiffnlp/twitter-roberta-base" #"roberta-large" #"bert-base-uncased" #"digitalepidemiologylab/covid-twitter-bert-v2" #"cardiffnlp/twitter-roberta-base" # "roberta-large" # "bert-base-uncased" # # #  # # # # #" # # #"digitalepidemiologylab/covid-twitter-bert-v2" #"roberta-large"#"bert-base-uncased" #"bert-large-cased" #"roberta-large" #"bert-base-uncased" #"bert-large-cased" # #"bert-base-uncased"
  tokenizer = AutoTokenizer.from_pretrained(checkpoint,truncation=True,max_length=512)   

  #Load the trained BERT model
  saved_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  saved_model.load_state_dict(torch.load(dir_path + "model_sydneysiege",map_location=torch.device('cpu')))
  #saved_model.to("cuda")

Some weights of the model checkpoint at vinai/bertweet-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'c

<All keys matched successfully>

We use a toy test set which contains 13 threads from the sydney-siege fold. 
Let's run the explainer model on a a particular example: take the first reply in the first twitter thread of the test sample. Run BERT on the concatenation between the source tweet and the reply.

In [39]:
 with open(dir_path +"explainer_input_test.json",'r') as f:
    test = json.load(f)

print(len(test['sydneysiege']))

print('Trial run for the explainer')

instance = test['sydneysiege'][0]
print(instance)
print('Claim: ', instance['sentence1'])
print('Reply: ', instance['replies1'][0])



13
Trial run for the explainer
{'replies1': ['@newscomauHQ I hope this is true', '@newscomauHQ lets hope they are ok.', '@newscomauHQ hope they are safe', 'BREAKING: Hostages are running out of the cafe #sydneysiege v @newscomauHQ', 'OMG is this true “@newscomauHQ: BREAKING: Hostages are running out of the cafe #sydneysiege”', '@newscomauHQ If this is true, what great news.  Is incident over?', '“@newscomauHQ: BREAKING: Hostages are running out of the cafe #sydneysiege” please let this be true', '“@newscomauHQ: BREAKING: Hostages are running out of the cafe #sydneysiege” I hope this is true', '@newscomauHQ Please let this be true!', '@newscomauHQ @mirandadevine if this is true, I fear this could turn very nasty very quickly as the remaining ones may be abused by terrorist', '@newscomauHQ how bout keeping that quiet @neontaster'], 'idx': 544350480780914688, 'labels': 0, 'sentence1': 'BREAKING: Hostages are running out of the cafe #sydneysiege'}
Claim:  BREAKING: Hostages are running out

We use the transformers-interpret library which relies on Integrated Gradients as laid out in the paper Axiomatic Attributions for Deep Networks (Sundararajan et al.,2017 https://arxiv.org/pdf/1703.01365.pdf). This method will ouput an attribution score for each token in the tokenized text.

In [17]:

#Find which replies contributed for the prediction
cls_explainer = MultiLabelClassificationExplainer(saved_model, tokenizer)
mini_text_input = instance['sentence1'] + ' [SEP] ' + instance['replies1'][0]
mini_text_input = preprocess_tweet(mini_text_input)
word_attributions = cls_explainer(mini_text_input,internal_batch_size=2)



In [18]:
print(word_attributions)

{'LABEL_0': [('<s>', 0.0), ('breaking', 0.02013541385752795), (':', -0.008238418946795948), ('hostages', 0.059403423359093446), ('are', 0.030279127155955345), ('running', -0.368238855365563), ('out', -0.10688875418590367), ('of', 0.15389767020172143), ('the', 0.23447527356990894), ('cafe', 0.21108824998935136), ('#', 0.12382871183519698), ('sy', 0.6533669436296611), ('d', -0.18311717640404573), ('neys', 0.16205672042505898), ('iege', 0.10915219608127644), ('[', -0.21827548559888135), ('se', 0.09449431965693357), ('p', 0.01769258883053469), (']', 0.08033574402856011), ('M', 0.3725545238239933), ('ENTION', 0.045396408027896365), ('i', -0.030684950591865295), ('hope', -0.0019691545662734815), ('this', -0.003057696333650379), ('is', -0.04093257733950469), ('true', -0.06384394519442635), ('</s>', 0.0)], 'LABEL_1': [('<s>', 0.0), ('breaking', 0.12755099058708746), (':', -0.0006147288380746749), ('hostages', -0.028348606078918894), ('are', -0.11490996673405893), ('running', -0.101931283297105

In [19]:
cls_explainer.visualize("multilabel_viz.html")

As seen in the visualisation above, the input's final attribution score is the sum of all individual attribution tokens. This is the highest for LABEL_0 which is also the predicted label for this instance.

In [20]:
gold_label = 'LABEL_' + str(instance['labels'])
predicted_label = cls_explainer.predicted_class_name
print("Predicted label ",predicted_label)
print("Gold Standard: ", gold_label)


Predicted label  LABEL_0
Gold Standard:  LABEL_0


However, we notice that different rationales (subsets of the input) may support other labels and thus contribute negatively to the prediction. 
In our case, we are interested to check which label the tokens correponding to the reply support the most (reply = rationale). We use the separator [SEP] to indentify the span of the reply text from the claim text.

For this example, we find that the reply "I hope this is true" will contribute positively to the overall prediction.

In [21]:
# Indentify which label the reply contributes too the most.

max_score = -1000
supporting_label = ''
start = 0
for label in word_attributions.keys():
    #Find position of separator token
    for i,x in enumerate(word_attributions[label]):
      if ']' in x[0] and i>1 and 'p' in word_attributions[label][i-1] and 'se' in word_attributions[label][i-2]:
        start = i
        break
    #Check contribution score of the reply's specific location
    reply_contribution = sum([word[1] for word in word_attributions[label][start+1:]])
    if reply_contribution>max_score:
        max_score = reply_contribution
        supporting_label = label

print('Label supported by the reply: ', supporting_label)

Label supported by the reply:  LABEL_0


Now apply this step to all replies in the thread. This process creates a partition of the replies with respect to each supported label. 

In [22]:
#Now apply this step to all replies in the thread. We will find a partition of the replies wrt to each supported label

cls_explainer = MultiLabelClassificationExplainer(saved_model, tokenizer)
contributions_label = {'LABEL_0':[],'LABEL_1':[],'LABEL_2':[]}
predicted_labels = []

for reply in instance['replies1']:

  mini_text_input = instance['sentence1'] + ' [SEP] ' + reply
  mini_text_input = preprocess_tweet(mini_text_input)
  cls_explainer = MultiLabelClassificationExplainer(saved_model, tokenizer)
  word_attributions = cls_explainer(mini_text_input,internal_batch_size=2)
  
  # For each source+reply, the model will predict a label. This might not be a very good indicator of reply support since the source tweet dictates the annotation
  predicted_label = cls_explainer.predicted_class_name
  predicted_labels.append(predicted_label)
  print(predicted_label)
    
  #___We want to find the label that this reply contributes too the most___
  max_score = -1000
  supporting_label = ''

  for label in contributions_label.keys():
      #Find position of separator token
      for i,x in enumerate(word_attributions[label]):
        if ']' in x[0] and i>1 and 'p' in word_attributions[label][i-1] and 'se' in word_attributions[label][i-2]:
          start = i
          break
      #Check contribution score of the reply's specific location
      reply_contribution = sum([word[1] for word in word_attributions[label][start+1:]])
      if reply_contribution>max_score:
          max_score = reply_contribution
          supporting_label = label
          
  contributions_label[supporting_label].append({'text':reply,'word_attr':word_attributions})

dict_to_save = {'replies':contributions_label, 'predicted_labels':predicted_labels,'source':instance['sentence1'],'gold_label':gold_label}

LABEL_0
LABEL_0
LABEL_0
LABEL_0
LABEL_0
LABEL_0
LABEL_0
LABEL_0
LABEL_0
LABEL_0
LABEL_0


As calculating integrated gradients occurs over several steps (default 50) and all replies within the thread, the construction of the word attribution dictionaries can be quite long depending on the computing resources used. 
Files containing the explainer outputs are are provided in .json format in the folder *explainer_output* for each thread in the toy test set. These dictionaries are organised as:



*   'replies': contains a dictionary of reply contributions for each label
  
*   'predicted labels': contains the predicted labels for each (source, reply) pair in the thread. Note that most predicted labels are consistent across all replies.
*   'source': text of source (equivalent to rumour claim)
*   'gold_label': the veracity of the source claim






# 3. Use scored rationales to create explanation-summary

In [34]:
#Load summarisation model

model_name_ft = '/content/gdrive/MyDrive/PhD/Code/mos_workspace/BART_model'
model_ft = BartForConditionalGeneration.from_pretrained(model_name_ft)
tokenizer_ft = AutoTokenizer.from_pretrained(model_name_ft)


def bart_ft(text,max_length,min_length):
    inputs = tokenizer_ft.encode(text, return_tensors="pt", max_length=1024)
    outputs = model_ft.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True,no_repeat_ngram_size=4)

    summary = tokenizer_ft.decode(outputs[0],skip_special_tokens=True)
    return summary



## Summarise content of Twitter thread which supports the predicted label.

From the previous step, we have indentified the replies supporting the prediction. Use the concatenation of the claim text (for context) and these replies as the input for the summarisation step.

In [40]:
# Heuristic to use for determining the aggregated predicted label of the Twitter thread: We choose the most frequent label. 
from collections import Counter

def most_common(mylist):
    data = Counter(mylist)
    return data.most_common(1)[0][0]

In [ ]:
# Run this cell if you want to obtain the explanation dictionary dict_to_save from the previous section. 
with open(dir_path + 'explainer_output/544350480780914688.json','r') as f:
  dict_to_save2 = json.load(f)

In [35]:
#Preprocess the summarisation input

text = [dict_to_save['source']]
text.extend([x['text'] for x in dict_to_save['replies'][predicted_label]])
text = ' '.join(text)
text = preprocess_tweet(text)
print('Input for Summarisation: ', text)

Input for Summarisation:  breaking: hostages are running out of the cafe #sydneysiege MENTION i hope this is true MENTION lets hope they are ok. MENTION hope they are safe breaking: hostages are running out of the cafe #sydneysiege v MENTION omg is this true MENTION: breaking: hostages are running out of the cafe #sydneysiege MENTION if this is true, what great news. is incident over? MENTION: breaking: hostages are running out of the cafe #sydneysiege please let this be true MENTION: breaking: hostages are running out of the cafe #sydneysiege i hope this is true MENTION please let this be true! MENTION how bout keeping that quiet MENTION


In [38]:
print(bart_ft(text,50,25))

 hostages are reportedly running out of the Sydney siege cafe The majority are hoping that this is true and are concerned for the safety of the hostages inside.


The summary successfully explains why the claim is predicted to be true as the majority of users express hope for the good news regarding the escape of some hostages.

## Summarise content of Twitter thread which supports the gold standard.

In the cases where the model makes an incorrect prediction, we can choose to provide a justification for the correct answer. 

We select an example of a thread (found in *explainer_output/544338575240609792.json*) where a false rumour is predicted as true. Despite the incorrect final judgement of the RV step, the explainer model correctly uncovers the replies which support the gold standard. 

This is in line with published work (Ma et al, ACL 2017; Li et al, EMNLP 2019; Bian et al,AAAI 2020) that found user interactions valuable for determining the veracity of rumours on social media.

In [47]:
with open(dir_path + 'explainer_output/544338575240609792.json','r') as f:
  dict_to_save2 = json.load(f)

gold_label = dict_to_save2['gold_label']
predicted_label = most_common(dict_to_save2['predicted_labels'])
print('Claim: ', dict_to_save2['source'])

print('Gold standard: ',gold_label)
print('Predicted_label: ',predicted_label)

for label in dict_to_save2['replies'].keys():
  print('#of replies supporting ',label,':', len(dict_to_save2['replies'][label]))

Claim:  #BREAKING: Flag w/ Islamic writing held up to window of #Sydney cafe. Police say up to 50 hostages. #Australia PM convenes security team.
Gold standard:  LABEL_1
Predicted_label:  LABEL_0
#of replies supporting  LABEL_0 : 1
#of replies supporting  LABEL_1 : 12
#of replies supporting  LABEL_2 : 4


In [48]:
#Preprocess the summarisation input

text = [dict_to_save2['source']]
text.extend([x['text'] for x in dict_to_save2['replies'][gold_label]])
text = ' '.join(text)
text = preprocess_tweet(text)
print('Input for Summarisation: ', text)

Input for Summarisation:  #breaking: flag w/ islamic writing held up to window of #sydney cafe. police say up to 50 hostages. #australia pm convenes security team. MENTION MENTION it's the shahadah flag, not the isis flag. MENTION MENTION 10 employees + ?how many customers = ? need verify? MENTION MENTION worth checking this out from MENTION on the flag itself. URL MENTION MENTION that flag is a shahadah flag. bandana on suspect says "at your service, o muhammad" URL MENTION MENTION nm - bloody sends electric waves to girls for physical relationships despite having own wife, extrema characterless MENTION MENTION islamic isn't a language MENTION MENTION cnn badly needs to employ reporters who can read. why not say as it is, that it's the islamic flag with shahada on it? MENTION MENTION breaking?? it's been happening for 3 hours now. sheesh. MENTION MENTION gee wonder why no one has a gun. oh that's right. australia bans citizens from having guns. MENTION MENTION #breaking ? its 5 hours 

In [57]:
print(bart_ft(text,50,25))

A flag with Islamic text written on it is held up to a window of a Sydney cafe, apparently holding hostages hostage. The majority believe that it is the Shahadah flag, not the isis flag.


The summary contains a justification for why the claim might be unreliable: the fact that it is a shahada flag and not an isis flag. However, the summary does not address the fact the rumour is no longer 'breaking news', nor that its number of reported hostages might be incorrect.

For a different summary, it is also possible to shuffle the replies in the summarisation input according to several criteria: support score to the label, similarity to the claim, time stamp etc. Below is an implementation of sorting wrt to support scores.

In [63]:
ordered_replies = []
for reply in dict_to_save2['replies'][gold_label]:
  for i,x in enumerate(reply['word_attr'][gold_label]):
    if ']' in x[0] and i>1 and 'p' in reply['word_attr'][gold_label][i-1] and 'se' in reply['word_attr'][gold_label][i-2]:
      start = i
      break
      
  reply_score = sum([word[1] for word in reply['word_attr'][gold_label][start+1:]]) / sum([word[1] for word in reply['word_attr'][gold_label]])
  ordered_replies.append((reply['text'],reply_score))

ordered_replies.sort(key= lambda x: x[1],reverse=True)
print(ordered_replies)

[("@willripleyCNN @CNN Gee wonder why no one has a gun. Oh that's right. Australia bans citizens from having guns.", 2.4309663223162943), ("@willripleyCNN @CNN CNN badly needs to employ reporters who can read. Why not say as it is, that it's the Islamic flag with Shahada on it?", 0.944405505035455), ('@willripleyCNN @CNN That flag is a Shahadah flag. Bandana on suspect says "At your service, O Muhammad" http://t.co/4HRFDy4jwE', 0.6537335932273545), ("@willripleyCNN @CNN Breaking?? It's been happening for 3 hours now. Sheesh.", 0.2871692694077432), ('@willripleyCNN @CNN NM - bloody sends electric waves to girls for physical relationships despite having own wife, extrema characterless', 0.2250301049290929), ('@willripleyCNN @SteveHuff Worth checking this out from @smh on the flag itself. http://t.co/ORzzRkKuCx', 0.22466616719173943), ("@willripleyCNN @CNN It's the Shahadah flag, not the ISIS flag.", 0.22207233265118018), ('@willripleycnn @cnn #Breaking ?  Its 5 hours into the #SydneySieg

In [64]:
text = [dict_to_save2['source']]
text.extend([x[0] for x in ordered_replies])
text = ' '.join(text)
text = preprocess_tweet(text)
print('Input for Summarisation: ', text)

Input for Summarisation:  #breaking: flag w/ islamic writing held up to window of #sydney cafe. police say up to 50 hostages. #australia pm convenes security team. MENTION MENTION gee wonder why no one has a gun. oh that's right. australia bans citizens from having guns. MENTION MENTION cnn badly needs to employ reporters who can read. why not say as it is, that it's the islamic flag with shahada on it? MENTION MENTION that flag is a shahadah flag. bandana on suspect says "at your service, o muhammad" URL MENTION MENTION breaking?? it's been happening for 3 hours now. sheesh. MENTION MENTION nm - bloody sends electric waves to girls for physical relationships despite having own wife, extrema characterless MENTION MENTION worth checking this out from MENTION on the flag itself. URL MENTION MENTION it's the shahadah flag, not the isis flag. MENTION MENTION #breaking ? its 5 hours into the #sydneysiege how is that breaking? MENTION MENTION islamic isn't a language MENTION MENTION hang tho

In [65]:
print(bart_ft(text,50,25))

A flag with Islamic text written on it is held up to a window of a Sydney cafe, apparently holding hostages hostage. The majority are shocked and confused about how this could possibly be happening.
